# Time projection of true tracks

Using masks and true track labels to create a 3D representation of cell evolution over time lapse

In [22]:
import napari
from macrohet import dataio, tile, visualise, tools
import btrack
print(btrack.__version__)
import os
import json 
import numpy as np
from tqdm.auto import tqdm

scale_factor = 5.04
ndim = 2

0.6.1.dev30


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Define row and column of choice

In [4]:
row = 3
column = 5

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [6]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane='sum_proj',
#                              set_channel=1,
#                              set_time = 66,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()

In [7]:
%%time
images = images.compute().compute()

CPU times: user 2h 49min 24s, sys: 19min 11s, total: 3h 8min 35s
Wall time: 7min 44s


In [8]:
images.shape

(75, 2, 6048, 6048)

# Load tracks

In [9]:
tracks_fn = os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}.h5')
with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
    tracks = hdf.tracks
    segmentation = hdf.segmentation
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=ndim)

[INFO][2023/06/01 11:17:46 AM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5...
[INFO][2023/06/01 11:17:46 AM] Loading tracks/obj_type_1
[INFO][2023/06/01 11:17:46 AM] Loading LBEP/obj_type_1
[INFO][2023/06/01 11:17:46 AM] Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
[INFO][2023/06/01 11:18:03 AM] Loading segmentation (75, 6048, 6048)
[INFO][2023/06/01 11:18:03 AM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5


In [12]:
gt_track_dict_fn = '/mnt/DATA/macrohet/upstream_development/tracking/tracking_performance/3,5/ground_truth_tracks/(3, 5)_track_assessment.json'
# Load the JSON data from the file
with open(gt_track_dict_fn, 'r') as file:
    gt_track_dict = json.load(file)

In [85]:
true_IDs = [int(ID) for ID, status in gt_track_dict.items() if status is True]
true_tracks = [t for t in tracks if t.ID in true_IDs] #and len(t) == 75]

In [65]:
false_IDs = [ID for ID, status in gt_track_dict.items() if status is not True]
false_IDs = list(set(int(x) for item in false_IDs for x in item.split(',')))
false_tracks = [t for t in tracks if t.ID in false_IDs]

In [76]:
# alternative method
false_tracks = [t for t in tracks if t.ID not in true_IDs]

In [87]:
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(true_tracks, ndim=ndim)

In [77]:
false_napari_tracks, false_properties, false_graph = btrack.utils.tracks_to_napari(false_tracks, ndim=ndim)

Rendering frames...


100%|████████████████████████████████████████████████████████| 100/100 [00:14<00:00,  6.96it/s]


### Recolour segmentation

In [88]:
%%time
col_segmentation = btrack.utils.update_segmentation(segmentation, true_tracks, scale = (scale_factor,)*ndim)

CPU times: user 33.2 s, sys: 36.1 s, total: 1min 9s
Wall time: 1min 9s


In [78]:
%%time
false_col_segmentation = btrack.utils.update_segmentation(segmentation, false_tracks, scale = (scale_factor,)*ndim)

CPU times: user 35.5 s, sys: 55.4 s, total: 1min 30s
Wall time: 1min 31s


### Binarise segmentation

So that I can convolve the images with true tracks - don't need to use typical approach as doesn't matter if segments are merged

In [89]:
%%time
binary_segmentation = col_segmentation>0

CPU times: user 885 ms, sys: 11 s, total: 11.9 s
Wall time: 11.9 s


In [90]:
gfp = images[:,0,...]
rfp = images[:,1,...]

In [91]:
gfp.shape

(75, 6048, 6048)

In [92]:
%%time
filtered_gfp = gfp*binary_segmentation

CPU times: user 1.19 s, sys: 6.62 s, total: 7.81 s
Wall time: 7.98 s


In [93]:
filtered_rfp = rfp*binary_segmentation

In [94]:
filtered_images = np.stack([filtered_gfp, filtered_rfp], axis = 1)

### Binarise false tracks

In [79]:
%%time
false_binary_segmentation = false_col_segmentation>0

CPU times: user 857 ms, sys: 319 ms, total: 1.18 s
Wall time: 1.17 s


In [80]:
false_filtered_gfp = gfp*false_binary_segmentation

In [81]:
false_filtered_rfp = rfp*false_binary_segmentation

In [82]:
false_filtered_images = np.stack([false_filtered_gfp, false_filtered_rfp], axis = 1)

In [73]:
false_filtered_images.shape

(75, 2, 6048, 6048)

# Launch napari image viewer

In [58]:
viewer = napari.Viewer()

viewer.add_image(filtered_images, 
                 channel_axis=1,
                 name=["macrophage", "mtb"],
                 colormap=["green",  "magenta"],
#                  contrast_limits=[[100, 6000], [100, 2000]],
                    scale=(100, 1, 1,), 

                 contrast_limits=[[0,450], [0,450]], 
                 visible = True
                 )

viewer.add_tracks(napari_tracks, scale = (100,scale_factor,scale_factor),
                    properties=properties, 
                    graph=graph, 
#                     name="Properly downscaled tracks", 
#                     blending="translucent",
#                     visible=True,
#                     scale = (100,1,1)
                 )



/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Tracks layer 'napari_tracks' at 0x7f445aa47ca0>

Rendering frames...


100%|████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.43it/s]


In [74]:
viewer = napari.Viewer()

viewer.add_image(false_filtered_images, 
                 channel_axis=1,
                 name=["macrophage", "mtb"],
                 colormap=["green",  "magenta"],
#                  contrast_limits=[[100, 6000], [100, 2000]],
                    scale=(100, 1, 1,), 

                 contrast_limits=[[0,450], [0,450]], 
                 visible = True
                 )

viewer.add_tracks(false_napari_tracks, scale = (100,scale_factor,scale_factor),
                    properties=false_properties, 
                    graph=false_graph, 
#                     name="Properly downscaled tracks", 
#                     blending="translucent",
#                     visible=True,
#                     scale = (100,1,1)
                 )

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Tracks layer 'false_napari_tracks' at 0x7f445ce39a00>

In [95]:
viewer = napari.Viewer()

viewer.add_image(filtered_images, 
                 channel_axis=1,
                 name=["macrophage", "mtb"],
                 colormap=["green",  "magenta"],
#                  contrast_limits=[[100, 6000], [100, 2000]],
                    scale=(100, 1, 1,), 
                    blending="additive",
                 contrast_limits=[[0,450], [0,450]], 
                 visible = True
                 )

viewer.add_tracks(napari_tracks, scale = (100,scale_factor,scale_factor),
                    properties=properties, 
                    graph=graph, 
#                     name="Properly downscaled tracks", 
                    blending="additive",#                     visible=True,
#                     scale = (100,1,1)
                 )

viewer.add_image(false_filtered_images, 
                 channel_axis=1,
                 name=["macrophage", "mtb"],
                 colormap=["cyan",  "magenta"],
#                  contrast_limits=[[100, 6000], [100, 2000]],
                    scale=(100, 1, 1,), 
                    blending="additive",
                 contrast_limits=[[0,450], [0,450]], 
                 visible = True
                 )

viewer.add_tracks(false_napari_tracks, scale = (100,scale_factor,scale_factor),
                    properties=false_properties, 
                    graph=false_graph, 
#                     name="Properly downscaled tracks", 
                    blending="additive",
#                     visible=True,
#                     scale = (100,1,1)
                 )

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Tracks layer 'false_napari_tracks' at 0x7f4464071f70>

Rendering frames...


100%|████████████████████████████████████████████████████████| 100/100 [00:18<00:00,  5.53it/s]


Rendering frames...


100%|████████████████████████████████████████████████████████| 100/100 [00:16<00:00,  6.05it/s]


Rendering frames...


100%|████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.69it/s]


Rendering frames...


100%|████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.64it/s]
